In [2]:
#conda install -c anaconda py-xgboost
# pip install xgboost
from xgboost import XGBClassifier

In [3]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
#import ipynb.fs.full.data_preprocessing_after_imputing as datapre

In [4]:
def best_xgboost(datatrain, datavali, parameter_space,criteria='accuracy', randomstate=0):
    '''
    if not specified, the parameters are n_estimators=100, criterion="gini", max_depth=8, min_samples_split=400, 
    min_samples_leaf=50, bootstrap=True, oob_score=False, warm_start=False, max_samples=3000, random_state=0
    '''
    from sklearn.model_selection import PredefinedSplit
    import numpy as np
    from xgboost.sklearn import XGBClassifier
    
    frames=pd.concat([datatrain, datavali])
    frames_reind=frames.set_index([pd.Index(range(len(frames)))])
    split_index=[-1]*len(datatrain)+[0]*len(datavali)
    pds = PredefinedSplit(test_fold = split_index)
    datadummy=pd.get_dummies(frames_reind)
    datatrainX=np.array(datadummy.drop('Default_ind', axis=1))
    datatrainy=datadummy['Default_ind']
    
    XGB=XGBClassifier()
    from sklearn.model_selection import GridSearchCV
    
    modelGS=GridSearchCV(XGB, parameter_space,cv=pds,scoring=criteria)
    modelGS.fit(datatrainX, datatrainy)
    
    return modelGS,datatrainX,datatrainy

In [5]:
def Cate_to_object_excp_inq(dataset):
    return(dataset.astype({'non_mtg_acc_past_due_12_months_num': 'object','non_mtg_acc_past_due_6_months_num': 'object','mortgages_past_due_6_months_num': 'object','card_inq_24_month_num': 'float64','inq_12_month_num': 'float64','card_open_36_month_num': 'object','auto_open_36_month_num': 'object','ind_acc_XYZ': 'object'}))

In [6]:
def upsample(data):
    from sklearn.utils import resample
    
    df_majority=data[data.Default_ind==0]
    df_minority=data[data.Default_ind==1]
    largenumber=len(df_majority)
    df_minority_upsampled=resample(df_minority,replace=True,n_samples=int(1.2*largenumber),random_state=123)
    df_upsampled=pd.concat([pd.DataFrame(df_majority),pd.DataFrame(df_minority_upsampled)])
    return df_upsampled

## Median Data

In [8]:
median_train=pd.read_csv('DataSet(Imputed)/Unconditionalmedianimpute/Unconditional_median_impute.csv')
median_test=pd.read_csv('DataSet(Imputed)/Unconditionalmedianimpute/Unconditional_median_impute_Test.csv')
median_validation=pd.read_csv('DataSet(Imputed)/Unconditionalmedianimpute/Unconditional_median_impute_Validation.csv')


In [9]:
#Change to categorical
cat_median_train=Cate_to_object_excp_inq(median_train)
cat_median_test=Cate_to_object_excp_inq(median_test)
cat_median_validation=Cate_to_object_excp_inq(median_validation)

In [10]:
# Upsample train and validation
cat_median_up_train=upsample(cat_median_train)
cat_median_up_validation=upsample(cat_median_validation)

In [11]:
datadummy=pd.get_dummies(cat_median_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [16]:
parameter_space = {
    'colsample_bytree': [0.8,0.9,1.0],
     'gamma': [0,0.5,1],
     'learning_rate': [0.2,0.3],
     'max_depth': [3,4,5],
     'min_child_weight': [0.5,1],
     'n_estimators': [30,40,50],
     'nthread': [4,7,9],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5,1],
    'use_label_encoder':[False]
    }

In [17]:
clf2,x2,y2=best_xgboost(cat_median_up_train,cat_median_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [18]:
y_pred=clf2.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix( testy,y_pred)

array([[3696,  903],
       [  88,  313]], dtype=int64)

In [19]:
clf2.best_params_

{'colsample_bytree': 0.8,
 'gamma': 0.5,
 'learning_rate': 0.3,
 'max_depth': 4,
 'min_child_weight': 0.5,
 'n_estimators': 30,
 'nthread': 4,
 'objective': 'reg:logistic',
 'scale_pos_weight': 1,
 'seed': 123,
 'subsample': 0.5,
 'use_label_encoder': False}

## EMB

In [20]:
EMB_train=pd.read_csv('DataSet(Imputed)/EMB_impute/EMB_impute.csv')
EMB_test=pd.read_csv('DataSet(Imputed)/EMB_impute/EMB_impute_Test.csv')
EMB_validation=pd.read_csv('DataSet(Imputed)/EMB_impute/EMB_impute_Vali.csv')

In [25]:
#Change to categorical
cat_emb_train=Cate_to_object_excp_inq(EMB_train)
cat_emb_test=Cate_to_object_excp_inq(EMB_test)
cat_emb_validation=Cate_to_object_excp_inq(EMB_validation)

In [26]:
# Upsample train and validation
cat_emb_up_train=upsample(cat_emb_train)
cat_emb_up_validation=upsample(cat_emb_validation)

In [27]:
datadummy=pd.get_dummies(cat_emb_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [28]:
parameter_space = {
    'colsample_bytree': [0.8,0.9,1.0],
     'gamma': [0,0.5,1],
     'learning_rate': [0.2,0.3],
     'max_depth': [3,4,5],
     'min_child_weight': [0.5,1],
     'n_estimators': [30,40,50],
     'nthread': [4,7,9],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5,1],
    'use_label_encoder':[False]
    }

In [29]:
clf3,x3,y3=best_xgboost(cat_emb_up_train,cat_emb_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [30]:
y_pred=clf3.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix( testy,y_pred)

array([[3688,  911],
       [  89,  312]], dtype=int64)

In [31]:
clf3.best_params_

{'colsample_bytree': 1.0,
 'gamma': 0,
 'learning_rate': 0.3,
 'max_depth': 3,
 'min_child_weight': 0.5,
 'n_estimators': 40,
 'nthread': 4,
 'objective': 'reg:logistic',
 'scale_pos_weight': 1,
 'seed': 123,
 'subsample': 0.5,
 'use_label_encoder': False}

## EM

In [8]:
EM_train=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/EM_imputed/Simulated_Data_Train_em.csv')
EM_train=EM_train.drop(EM_train.columns[0],axis=1)
EM_test=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/EM_imputed/Simulated_Data_Test_em.csv')
EM_test=EM_test.drop(EM_test.columns[0],axis=1)
EM_validation=pd.read_csv('/Users/shiyuandeng/Desktop/UGA/DataScienceCompetition/Data Set (Imputed)/EM_imputed/Simulated_Data_validatiion_em.csv')
EM_validation=EM_validation.drop(EM_validation.columns[0],axis=1)

In [9]:
#Change to categorical
cat_em_train=Cate_to_object_excp_inq(EM_train)
cat_em_test=Cate_to_object_excp_inq(EM_test)
cat_em_validation=Cate_to_object_excp_inq(EM_validation)

In [10]:
# Upsample train and validation
cat_em_up_train=upsample(cat_em_train)
cat_em_up_validation=upsample(cat_em_validation)

In [11]:
datadummy=pd.get_dummies(cat_em_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']

In [12]:
parameter_space = {
    'colsample_bytree': [0.8,0.9,1.0],
     'gamma': [0,0.5,1],
     'learning_rate': [0.2,0.3],
     'max_depth': [3,4,5],
     'min_child_weight': [0.5,1],
     'n_estimators': [30,40,50],
     'nthread': [4,7,9],
     'objective': ['reg:logistic'],
     'scale_pos_weight': [1],
     'seed': [123],
     'subsample': [0.5,1],
    'use_label_encoder':[False]
    }

In [13]:
clf4,x4,y4=best_xgboost(cat_em_up_train,cat_em_up_validation,parameter_space,criteria='accuracy', randomstate=0)

In [14]:
y_pred=clf4.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix( testy,y_pred)

array([[3921,  678],
       [ 130,  271]])

In [15]:
clf4.best_params_

{'colsample_bytree': 1.0,
 'gamma': 1,
 'learning_rate': 0.3,
 'max_depth': 5,
 'min_child_weight': 0.5,
 'n_estimators': 50,
 'nthread': 4,
 'objective': 'reg:logistic',
 'scale_pos_weight': 1,
 'seed': 123,
 'subsample': 0.5,
 'use_label_encoder': False}